In [37]:
import numpy as np

import matplotlib as mpl
import matplotlib.pyplot as plt

import os
#%matplotlib inline


In [38]:
from skimage import filters, transform, util

from scipy import ndimage as ndi
from scipy.spatial import distance_matrix

from skimage.morphology import watershed
from skimage.feature import peak_local_max

In [39]:
def remove_close_duplicates(points, max_distance=100):
    distances = distance_matrix(np.transpose(points), np.transpose(points))
    mask = (np.tril(np.ones(np.shape(distances)))*(max_distance+1))
    selection_matrix = (distances + mask)
    duplicates =(np.where(selection_matrix < max_distance)[1])
    return( np.delete(points, duplicates, 1) )

In [40]:
def crop_center(img, center,size):
    #img_width, img_length = np.shape(img)
    
    x,y = center
    top = int(y-(size/2.0))
    bottom = int(y+(size/2.0))
    left = int(x-(size/2.0))
    right = int(x+(size/2.0))
    
    return img[top:bottom,left:right]

In [41]:
def show_images(images):
    for image in images:
        plt.imshow(image)
        plt.show()

In [42]:
def extractDarkGrains(img, size=600, verbose =False):
    img_copy = img
    if verbose:
        plt.imshow(img_copy)
        plt.show()
    # Add margin to be able to take image of size even if grain is at the very edge of picture
    margin_size = int(1+(size/2))
    padding = ((margin_size,margin_size),(margin_size,margin_size),(0,0))
    img_copy = util.pad(img_copy, padding, constant_values=np.mean(img_copy))
    if verbose:
        plt.imshow(img_copy)
        plt.show()
    mask = img_copy

    # make it monochrome (green channel works the best in the case)
    mask = mask[:,:,1]
    if verbose:
        plt.imshow(mask)
        plt.show()

    # blurr to avoid holes in binarization
    blurred = filters.gaussian(mask, sigma=50)
    if verbose:
        plt.imshow(blurred)
        plt.show()

    #binarize - cutoff level relative to mean value to remove those pesky local maximums
    mask =  blurred<(np.mean(blurred)*0.74)
    if verbose:
        plt.imshow(mask)
        plt.show()
    
    #Watershed
    distance = ndi.distance_transform_edt(mask)
    local_maxi = peak_local_max(distance, indices=False, min_distance=1000,footprint=np.ones((50, 50)),
                                labels=mask)

    markers = ndi.label(local_maxi)[0]
    labels = watershed(blurred,markers=markers, mask=mask, compactness=0.5)

    if verbose:
        plt.imshow(distance)
        plt.show()
        plt.imshow(labels)
        plt.show()
    un_labs = np.unique(labels)
    if verbose:
        print(len(np.unique(labels)))
    sizes=[]
    for label in un_labs[1:]:
        sizes.append(np.sum(labels == label))

    # remove results smaller than arbitrary value (1/5 of average pollen size)
    relevant_indices = (np.where(sizes>np.float64(10000))[0])
    maxi_centers = np.where(local_maxi)
    relevant_centers = maxi_centers[0][relevant_indices],maxi_centers[1][relevant_indices],

    relevant_centers = remove_close_duplicates(relevant_centers)

    images = []
    
    # crop images 
    for i in range(0,len(relevant_centers[1])): 
        img_frame =crop_center(img_copy,(relevant_centers[1][i], relevant_centers[0][i]),size)
        if verbose:
            print( np.mean(np.abs(filters.rank.gradient(img_frame[:,:,1], np.ones((5,5))))))
            plt.imshow(img_frame)
            plt.show()
        images.append(img_frame)
        
    return(images)

In [43]:
# Load images of the same plant species
# Create the imgs directory for storing the segmented pollen images. 
# Fill it in with the images.
plant_names = ['cichorei', 'keizerskaars', 'phacelia', 'wondklaver']

files = os.listdir('..')
image_size = 600

if not os.path.exists('./imgs/'):
    os.makedirs('./imgs/')
    
for plant in plant_names: 
    i=1
    for f in files:
        if plant in f and 'jpg' in f:
            x = plt.imread('../'+f)
            y = extractDarkGrains(x, image_size)
            for im in y:
                mpl.image.imsave("imgs/"+plant+"_" + str(i) + '.png', im)
                i = i + 1

['cichorei 3.dng', 'cichorei2.jpg', 'cichorei.dng', 'keizerskaars.jpg', 'masterimage.py', 'Untitled.ipynb', 'cichorei2.dng', 'pollen-master', 'keizerskaars4.dng', 'process_microscopic_image.ipynb', 'cichorei.jpg', 'cichorei4.dng', 'phacelia.dng', 'keizerskaars.dng', 'phacelia.jpg', 'keizerskaars2.jpg', 'keizerskaars3.jpg', 'wondklaver.dng', 'cichorei-3.jpg', 'keizerskaars4.jpg', 'cichorei4.jpg', 'keizerskaars3.dng', 'keizerskaars2.dng', 'wondklaver.jpg', '.ipynb_checkpoints']
